In [ ]:
from concurrent.futures import ThreadPoolExecutor
import openai
import logging
import pandas as pd
import requests
from stocksymbol import StockSymbol

# Configuration
fmp_key = "API KEY"
news_api_key = "API KEY"
openai_api_key = "API KEY"

api_key = 'API KEY' #stock symbol API key
ss = StockSymbol(api_key)

# Initialize OpenAI
openai.api_key = openai_api_key

# Base URLs - two available as financialmodelingprep has varying api numbers for different metrics
base_url = "https://financialmodelingprep.com/api/v3"
base_urlv4 = "https://financialmodelingprep.com/api/v4"

# Set up logging
logging.basicConfig(filename='stock_selection.log', level=logging.INFO,
                    format='%(asctime)s %(levelname)s %(message)s')

# Stock symbol list - choose which market to retrieve symbol lists from
symbol_list_us = ss.get_symbol_list(market="US", symbols_only=True)


# Value Investing Thresholds
# Set your thresholds according to what you are looking for in a stock
price_to_earnings = 10
price_to_book = 1.5
debt_to_equity = 1
roe = 0.12

In [ ]:
# Fetch financial data for each ticket in the market you have selected
def get_financial_data(ticker):
    try:
        key_metrics_url = f"{base_url}/key-metrics-ttm/{ticker}?apikey={fmp_key}"
        key_metrics = requests.get(key_metrics_url).json()[0]

        historical_price_url = f"{base_url}/historical-price-full//{ticker}?apikey={fmp_key}"
        historical_price = requests.get(historical_price_url).json()

        income_statement_url = f"{base_url}/income-statement/{ticker}?apikey={fmp_key}"
        income_statement = requests.get(income_statement_url).json()[0]

        shares_float_url = f"{base_urlv4}/shares_float?symbol={ticker}&apikey={fmp_key}"
        shares_float = requests.get(shares_float_url).json()[0]

        ratios_url = f"{base_url}/ratios-ttm/{ticker}?&apikey={fmp_key}"
        ratios_data = requests.get(ratios_url).json()[0]

        return key_metrics, historical_price, income_statement, shares_float, ratios_data

    except Exception as e:
        logging.error(f"Error occurred while processing {ticker}: {e}")
        return None

In [ ]:
# Sentiment analysis on the stock using open AI
def sentiment_analysis(ticker):
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=f"Analyze recent news articles and social media posts about stock {ticker} and provide a sentiment "
               f"analysis. Determine if the overall sentiment is positive, negative, or neutral.",
        temperature=0.2,
        max_tokens=150,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
    )
    return response.choices[0].text.strip()

# Analyse earnings calls for the stock using open AI
def earnings_call(ticker):
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=f"Read the latest earnings call transcript for stock {ticker} and summarize the key points briefly, "
               f"including management's outlook, market conditions, and any potential risks or opportunities mentioned "
               f"during the call.",
        temperature=0.2,
        max_tokens=250,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
    )
    return response.choices[0].text.strip()

# Stock analysis using open AI
def stock_insights(ticker):
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=f"Provide an brief analysis of stock {ticker}, covering aspects such as its business model, "
               f"competitive landscape, growth prospects, financial performance, "
               f"and any potential risks or opportunities.",
        temperature=0.2,
        max_tokens=250,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
    )
    return response.choices[0].text.strip()

# Value investing analysis using open AI
def value_investing(ticker):
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=f"Analyze stock {ticker} from a value investor's perspective. Compare its key metrics, "
               f"such as Price-to-Earnings ratio, Price-to-Book ratio, and Return on Equity, to the industry average."
               f"Analyze the company's financial performance, competitive landscape, and growth prospects."
               f"Based on this analysis, evaluate whether the stock is a good investment opportunity "
               f"for a value investor.",
        temperature=0.2,
        max_tokens=250,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
    )
    return response.choices[0].text.strip()

In [ ]:
# Function to process each ticket and retrieve the metrics we want to review
# Print statements also included for de-bugging if needed
def process_stock(ticker):
    financial_data = get_financial_data(ticker)

    if financial_data is None:
        return None

    key_metrics, historical_price, income_statement, shares_float, ratios_data = financial_data

    try:
        market_price = historical_price['historical'][-1]['close']
    except KeyError:
        logging.error(f"Error: 'historical' key not found for {ticker}")
        return None
    # print("market price =", market_price)
    earnings_per_share = income_statement.get('eps')
    # print("eps = ", earnings_per_share)
    shares_outstanding = shares_float.get('outstandingShares')
    # print("shares_outstanding = ", shares_outstanding)
    book_value_per_share = key_metrics.get('bookValuePerShareTTM')
    # print("bookvalue per share= ", book_value_per_share)
    dividend_yield = key_metrics.get('dividendYieldTTM')
    # print("dividend yield= ", dividend_yield)
    price_to_earnings_ratio = key_metrics.get('peRatioTTM')
    # print("pte= ", price_to_earnings_ratio)
    price_to_book_ratio = key_metrics.get('ptbRatioTTM')
    # print("ptb= ", price_to_book_ratio)
    payout_ratio = key_metrics.get('payoutRatioTTM')
    # print("payout ratio= ", payout_ratio)
    debt_to_equity_ratio = key_metrics.get('debtToEquityTTM')
    # print("debt to eq ratio= ", debt_to_equity_ratio)
    roe_ratio = key_metrics.get('roeTTM')
    # print("roe ratio= ", roe_ratio)
    revenue_per_share = key_metrics.get('revenuePerShareTTM')
    # print("rev per share ratio= ", revenue_per_share)
    gross_profit_margin_ratio = ratios_data.get('grossProfitMarginTTM')
    # print("gross profit margin ratio= ", gross_profit_margin_ratio)
    price_to_sales_ratio = ratios_data.get('priceToSalesRatioTTM')
    # print("price_to_sales_ratio= ", gross_profit_margin_ratio)

    if (price_to_earnings_ratio is not None and price_to_book_ratio is not None and
            debt_to_equity_ratio is not None and roe_ratio is not None and
            price_to_earnings > price_to_earnings_ratio > 0 and
            price_to_book > price_to_book_ratio > 0 and
            debt_to_equity > debt_to_equity_ratio > 0 and
            roe_ratio > roe and roe_ratio > 0):
        sentiment_insight = sentiment_analysis(ticker)
        earnings_insight = earnings_call(ticker)
        stock_insight = stock_insights(ticker)
        value_insight = value_investing(ticker)

        return {
            'company': ticker,
            'market_price': market_price,
            'earnings_per_share': earnings_per_share,
            'book_value_per_share': book_value_per_share,
            'dividend_yield': dividend_yield,
            'shares_outstanding': shares_outstanding,
            'price_to_earnings_ratio': price_to_earnings_ratio,
            'price_to_book_ratio': price_to_book_ratio,
            'payout_ratio': payout_ratio,
            'debt_to_equity_ratio': debt_to_equity_ratio,
            'roe_ratio': roe_ratio,
            'revenue_per_share': revenue_per_share,
            'gross_profit_margin_ratio': gross_profit_margin_ratio,
            'price_to_sales_ratio': price_to_sales_ratio,
            'sentiment_insight': sentiment_insight,
            'earnings_insight': earnings_insight,
            'stock_insight': stock_insight,
            'value_insight': value_insight
        }

In [ ]:
# Main execution to create the final spreadsheet / output
# Will print out the number of tickers processed to help with debugging or to determine if script has timed outd
columns = ['company', 'market_price', 'earnings_per_share', 'book_value_per_share', 'dividend_yield',
           'shares_outstanding', 'price_to_earnings_ratio', 'price_to_book_ratio', 'payout_ratio',
           'debt_to_equity_ratio', 'roe_ratio', 'revenue_per_share', 'gross_profit_margin_ratio',
           'price_to_sales_ratio', 'sentiment_insight', 'earnings_insight', 'stock_insight', 'value_insight']

df_portfolio = pd.DataFrame(columns=columns)

total_tickers = len(symbol_list_us)
tickers_processed = 0
tickers_added = 0

# ThreadPoolExecutor to speed up the process
with ThreadPoolExecutor() as executor:
    results = executor.map(process_stock, symbol_list_us)

    for result in results:
        tickers_processed += 1
        print(f"Tickers processed: {tickers_processed}/{total_tickers}")
        if result is not None:
            tickers_added += 1
            print(f"Tickers added: {tickers_added}")
            df_portfolio = pd.concat([df_portfolio, pd.DataFrame([result], columns=columns)], ignore_index=True)
            # Write the new row to the CSV file
            df_portfolio.to_csv('/Users/directory.csv', index=False,
                                mode='a', header=(tickers_added == 1))

    # Save the final portfolio to a CSV file
    df_portfolio.to_csv('/Users/directory.csv', index=False)